# Discontinuous Galerkin Methods (Unit 2.8)

- Use discontinuous finite element spaces to solve PDEs.
- Allows upwind-stabilization for convection-dominated problems
- Requires additional jump terms for consistency

Interior penalty DG form for $−\Delta u$:

$$A(u,v)=\sum_T \int_T \nabla u \nabla v -\frac{1}{2}\sum_F \int_F \lbrace n\nabla v \rbrace [u] - \frac{1}{2}\sum_F \int_F \lbrace n\nabla u \rbrace [v] + \frac{\alpha p^2}{h} \sum_F \int_F [u][v]$$

with jump-term over facets:

$$[u]=u_{left}−u_{right}$$

$$\lbrace n \nabla u \rbrace =\frac{1}{2} (n_{left} \nabla u_{left} + n_{left} \nabla u_{right})$$

DG form for div $bu$, where $b$ is the given wind:
$$\text{added integral on lhs: } B(u,v)=−\sum_T \int_T u\ b \cdot \nabla v + \sum_F \int_F b \cdot n u^{upwind} v$$

## Notes:
It's not clear what problem we're trying to solve here.  It would be good to figure that out.  Maybe something like:
$$ - \Delta u + b \cdot \nabla u = f$$ with zero Dirichlet conditions. I think the $\frac{\alpha p^2}{h} \sum_F \int_F [u][v]$ term comes from the DG method.  In the example below, the source $f$ is constant=1

In [1]:
import netgen.gui
%gui tk
from netgen.geom2d import unit_square
from ngsolve import *
mesh = Mesh(unit_square.GenerateMesh(maxh=0.3))


The space is responsible for allocating the matrix graph. Tell it to reserve entries for the coupling terms:

In [2]:
order=4
fes = L2(mesh, order=order, dgjumps = True)
u,v = fes.TnT()

Every facet has a master element. The value from the other element is referred to via the Other() operator:

In [3]:
jump_u = u-u.Other()
jump_v = v-v.Other()
n = specialcf.normal(mesh.dim)
mean_dudn = 0.5*n * (grad(u)+grad(u.Other()))
mean_dvdn = 0.5*n * (grad(v)+grad(v.Other()))

In [4]:
mean_dudn.dim # reality check

1

Integrals on facets are computed by setting `skeleton=True`. This iterates over all internal facets. Additionally setting `BND` iterates only over boundary facets:

### Note:
It's not entirely clear when to use this vs. `element_boundary=True`.  Looking at the code in python_comp.cpp, it appears that `skeleton=True` causes a SymbolicFacetFormIntegrator instead of a SymbolicFormIntegrator to be used.  If we try `element_boundary=True` below we get: "Inconsistent number of degrees of freedom in Assemble BilinearForm 'biform_from_py low-order'"

Note that the space chosen is $L^2$, but we are still able to compute grad(u)

In [5]:
alpha = 4
h = specialcf.mesh_size
a = BilinearForm(fes)
a += SymbolicBFI(grad(u)*grad(v))                                    # term 1 in first equation
a += SymbolicBFI(alpha*order**2/h*jump_u*jump_v, skeleton=True)      # term 4 in first equation
a += SymbolicBFI(alpha*order**2/h*u*v, BND, skeleton=True)            # boundary term corresp to term 4
# BC - I think that what is going on here is that we're imposing (via the boundary facets) 
# the boundary conditon from integration by parts if we have dirichlet boundaries 
# I think we don't have jumps on the boundary (no 'other')

a += SymbolicBFI(-mean_dudn*jump_v -mean_dvdn*jump_u, skeleton=True) # terms 2 and 3 in first equation (factor of 1/2?)
a += SymbolicBFI(-n*grad(u)*v-n*grad(v)*u, BND, skeleton=True)       # boundary term corresp to terms 2 and 3
# BC - I think it doesn't make sense to compute mean values on the boundary (no 'other')

a.Assemble()

In [6]:
f = LinearForm(fes)
f += SymbolicLFI(1*v)  # constant source
f.Assemble()

In [7]:
gfu = GridFunction(fes, name="uDG")
gfu.vec.data = a.mat.Inverse() * f.vec
Draw (gfu)

DG requires a lot of additional matrix entries:

In [8]:
print ("a nze:", a.mat.nze) # nonzero entries?
fes2 = L2(mesh, order=order)
a2 = BilinearForm(fes2)
a2 += SymbolicBFI(u*v)
a2.Assemble()
print ("a2 nze:", a2.mat.nze)

a nze: 23400
a2 nze: 6750


### Notes: 
Notice that we did not specify any Dirichlet boundary conditions here, they were handled through the integrals.

What does $\alpha$ represent?  Why have we chosen its value to be 4?  Changing it to 40, 0.04 even 0 doesn't seem to change anything about the solution in this case.

We can omit either one of the boundary integral terms without significantly changing the result, but if we eliminate both, the solution becomes unbounded.

Omitting term 4 seems to have no effect at all on the solution
Omitting terms 2 and 3 seems to have negligible effect
Omitting both terms 4 and 2,3 causes the integration to fail
Omitting only term 1 also causes the integration to fail


### convection-diffusion problem
Next we are solving a convection-diffusion problem:

In [9]:
# Exactly as above
alpha = 4
h = specialcf.mesh_size
acd = BilinearForm(fes)
acd += SymbolicBFI(grad(u)*grad(v))
acd += SymbolicBFI(alpha*order**2/h*jump_u*jump_v, skeleton=True)
acd += SymbolicBFI(alpha*order**2/h*u*v, BND, skeleton=True)
acd += SymbolicBFI(-mean_dudn*jump_v -mean_dvdn*jump_u, skeleton=True)
acd += SymbolicBFI(-n*grad(u)*v-n*grad(v)*u, BND, skeleton=True)

The IfPos checks whether the first argument is positive. Then it returns the second one, else the third one. This is used to define the upwind flux. The check is performed in every integration-point on the skeleton:

In [10]:
b = CoefficientFunction( (20,1) )  # vector strongly to right, slightly up
acd += SymbolicBFI(-b * u * grad(v))              # term 1 in second equation above
uup = IfPos(b*n, u, u.Other())                    # u_upwind (upwind flux)
acd += SymbolicBFI(b*n*uup*jump_v, skeleton=True) # term 2 in second equation above

In [11]:
acd.Assemble()

In [12]:
gfu = GridFunction(fes)
gfu.vec.data = acd.mat.Inverse(freedofs=fes.FreeDofs(),inverse="umfpack") * f.vec
Draw (gfu)

### Notes:
We get the same behavior as above if we remove boundary integral terms. 
Omitting term 4 keeps the basic shape of the solution, but it now appears 'fractured'
Omitting terms 2 and 3 seems to have negligible effect
Omitting both terms 4 and 2,3 causes the integration to fail
Omitting only term 1 also causes the integration to fail

## Hybrid Discontinuous Galerkin methods
use additionally the hybrid facet variable on the skeleton:
$$A(u,\hat{u};v,\hat{v})=\sum_T \int_T \nabla u \nabla v - \sum_T \int_{\partial_T} n \nabla u (v - \hat{v})  - \sum_T \int_{\partial_T} n \nabla u (u - \hat{u}) + \frac{\alpha p^2}{h} \sum_F \int_F (u-\hat{u})(v - \hat{v})$$

the jump-term is now replaced by the difference $u−\hat{u}$.

No additional matrix entries across elements are produced. Dirichlet boundary conditions are set as usual to the facet variable:


In [13]:
order=4
V = L2(mesh, order=order)
F = FacetFESpace(mesh, order=order, dirichlet="bottom|left|right|top")
fes = FESpace([V,F])
u,uhat = fes.TrialFunction()
v,vhat = fes.TestFunction()

Now, the jump is the difference between element-term and facet-term:

In [14]:
jump_u = u-uhat
jump_v = v-vhat

In [17]:
alpha = 4
condense = True
h = specialcf.mesh_size
n = specialcf.normal(mesh.dim)

a = BilinearForm(fes, eliminate_internal=condense)
a += SymbolicBFI(grad(u)*grad(v))
a += SymbolicBFI(alpha*order**2/h*jump_u*jump_v, element_boundary=True)
a += SymbolicBFI(-grad(u)*n*jump_v - grad(v)*n*jump_u, element_boundary=True)

b = CoefficientFunction( (20,1) )
a += SymbolicBFI(-b * u * grad(v))
uup = IfPos(b*n, u, uhat)
a += SymbolicBFI(b*n*uup*jump_v, element_boundary=True)
a.Assemble()

f = LinearForm(fes)
f += SymbolicLFI(1*v)
f.Assemble()

gfu = GridFunction(fes)

print ("A non-zero elements:", a.mat.nze)

A non-zero elements: 5825


In [18]:
if not condense:
    inv = a.mat.Inverse(fes.FreeDofs(), "umfpack")
    gfu.vec.data = inv * f.vec
else:
    f.vec.data += a.harmonic_extension_trans * f.vec 
    
    inv = a.mat.Inverse(fes.FreeDofs(True), "umfpack")
    gfu.vec.data = inv * f.vec
    
    gfu.vec.data += a.harmonic_extension * gfu.vec
    gfu.vec.data += a.inner_solve * f.vec

Draw (gfu.components[0], mesh, "u-HDG")


## Notes:
The solution is nearly identical to the previous solutions.  We had to introduce another space, but I think the matrix is smaller

